# Jupyter Notebook example for preparing subcatalogues

In [1]:
import numpy as np
import h5py
import pickle

# Load OQ tools
from openquake.hmtk.parsers.catalogue import CsvCatalogueParser
from openquake.hmtk.seismicity.selector import CatalogueSelector
from openquake.hmtk.parsers.catalogue.csv_catalogue_parser import CsvCatalogueWriter  

In [8]:
# Configuration files
cat_pickle_filename = '~/model/catalogue/csv/catalogue.pkl'
treg = '~/model/catalogue/classification/classified.hdf5'

In [9]:
# Reading TR hdf5 file and creating the list of tectonic regions
aaa = []
f = h5py.File(treg, "r")
for key in f.keys():
    aaa.append(key)
    alen = len(f[key])
    print(key)
f.close()

crustal
crustal_deep
int_prt
slab_nht
slab_prt


In [10]:
# for each label, create the subcatalogue
tot_lab = np.zeros(alen)
for label in (aaa):
    csv_filename = "cat_TR_%s.csv"%(label)
    f = h5py.File(treg,'r')
    tr = f[label][:]
    f.close()
    if sum(tr) > 0:
        tmp_lab = tr*1
        tot_lab = tot_lab+tmp_lab
        catalogue = pickle.load(open(cat_pickle_filename, 'rb'))
        for lab in ['month', 'day', 'hour', 'minute', 'second']:
            idx = np.isnan(catalogue.data[lab])
            if lab == 'day' or lab == 'month':
                catalogue.data[lab][idx] = 1
            elif lab == 'second':
                catalogue.data[lab][idx] = 0.0
            else:
                catalogue.data[lab][idx] = 0
        selector = CatalogueSelector(catalogue, create_copy=False)
        print('# earthquakes in the catalogue: {:d}'.format(len(catalogue.data['longitude'])))
        catalogue = selector.select_catalogue(tr)
        
        print('# earthquakes in this TR      : {:d}'.format(len(catalogue.data['longitude'])))
        # Sub-catalogue
        csvcat = CsvCatalogueWriter(csv_filename) 
        # Write the purged catalogue
        csvcat.write_file(catalogue)
        print("Catalogue successfully written to %s" % csv_filename)

# earthquakes in the catalogue: 16553
# earthquakes in this TR      : 10999
Catalogue successfully written to cat_TR_crustal.csv
# earthquakes in the catalogue: 16553
# earthquakes in this TR      : 1212
Catalogue successfully written to cat_TR_crustal_deep.csv
# earthquakes in the catalogue: 16553
# earthquakes in this TR      : 1933
Catalogue successfully written to cat_TR_int_prt.csv
# earthquakes in the catalogue: 16553
# earthquakes in this TR      : 626
Catalogue successfully written to cat_TR_slab_nht.csv
# earthquakes in the catalogue: 16553
# earthquakes in this TR      : 296
Catalogue successfully written to cat_TR_slab_prt.csv


In [11]:
# also make a catalogue of unclassified earthquakes
tr_undef = abs(tot_lab-1)
catalogue = pickle.load(open(cat_pickle_filename, 'rb'))
selector = CatalogueSelector(catalogue, create_copy=False)
print('# earthquakes: {:d}'.format(len(catalogue.data['longitude'])))
catalogue = selector.select_catalogue(tr_undef)
print('# earthquakes: {:d}'.format(len(catalogue.data['longitude'])))
# Sub-catalogue
csv_filename = "cat_TR_unclassified.csv"
csvcat = CsvCatalogueWriter(csv_filename) 
# Write the purged catalogue
csvcat.write_file(catalogue)
print("Catalogue successfully written to %s" % csv_filename)

# earthquakes: 16553
# earthquakes: 1487
Catalogue successfully written to cat_TR_unclassified.csv
